In [1]:
# test to see if we can load these in buildingmotif 
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library, Model

bm = BuildingMOTIF("sqlite://")
templates = Library.load(directory='s223-test-templates')

/Users/lazlopaul/Desktop/HPflex/semantic_objects/.venv/lib/python3.11/site-packages/pyshacl/extras/__init__.py:46: Warning: Extra "js" is not satisfied because requirement pyduktape2 is not installed.
  warn(Warning(f"Extra \"{extra_name}\" is not satisfied because requirement {req} is not installed."))


In [2]:
ex_template_dir = "/Users/lazlopaul/Desktop/HPflex/Semantic_MPC_Interface/src/semantic_mpc_interface/templates/brick-templates"

In [3]:
ex_templates = Library.load(directory=ex_template_dir)

In [4]:
templates.get_templates()

[]

In [5]:
ex_templates.get_templates()

[Template(_id=1, _name='hvac-zone', body=<Graph identifier=5b3d6c2f-172d-49cb-b8cc-9fe14b0cdbf0 (<class 'rdflib.graph.Graph'>)>, optional_args=['space', 'window', 'tstat', 'hvac'], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=2, _name='space', body=<Graph identifier=0a5c2d9d-6c2a-4a89-8973-7ffa82dbf2f5 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=3, _name='window', body=<Graph identifier=7eb8fcef-af22-4ed2-b667-7ed84816b16a (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=4, _name='hp-rtu', body=<Graph identifier=b3444f99-dc44-4b50-b5da-59739754b078 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=5, _name='tstat', body

In [6]:
ex_template_2 = Library.load(directory = 'ex-templates')

In [7]:
ex_template_2.get_templates()

[Template(_id=45, _name='area', body=<Graph identifier=1a0aa440-a25d-41a3-9834-5c8576d5e0cf (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=46, _name='tilt', body=<Graph identifier=1f304be5-345c-4668-9c8b-20a628e76731 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=47, _name='azimuth', body=<Graph identifier=017ac19b-f132-45f4-8213-9a2c28a67ec0 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=48, _name='cooling-COP', body=<Graph identifier=dd4bf7f4-bfaf-441d-b3f4-03c3d4b5c9d2 (<class 'rdflib.graph.Graph'>)>, optional_args=[], _bm=<buildingmotif.building_motif.building_motif.BuildingMOTIF object at 0x11a07f910>),
 Template(_id=49, _name='heating-COP', body=<Graph identifier=0091